In [1]:
!pip install python-telegram-bot

     |████████████████████████████████| 378kB 2.8MB/s 
     |████████████████████████████████| 2.6MB 13.6MB/s 


In [2]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import numpy as np
import urllib
from urllib import request
import os
import cv2
from google.colab import drive
from tensorflow.keras.models import load_model

In [3]:
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/project2/")

Mounted at /content/drive/


In [4]:
token = '1342350093:AAHujf5nQB-O6bOlPzWVu79G3-gDAkxBRiQ'

In [5]:
updater = Updater(token=token, use_context=True)

In [6]:
dispatcher = updater.dispatcher

In [7]:
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text="I'm a bot, please send me your pic!")

In [8]:
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [9]:
def image_url_to_numpy_array_urllib(url, format=None):
    # read as HTTPResponse 
    resp = urllib.request.urlopen(url)
    ## read as 1D bytearray
    resp_byte_array = resp.read()
    ## returns a bytearray object which is a mutable sequence of integers in
    # the range 0 <=x< 256
    mutable_byte_array = bytearray(resp_byte_array)
    ## read as unsigned integer 1D numpy array
    image = np.asarray(mutable_byte_array, dtype="uint8")
    ## To decode the 1D image array into a 2D format with RGB color components
    # we make a call to cv2.imdecode
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # filter images with too rectangular ratio
    if (image.shape[0]>(2*image.shape[1]))or(image.shape[1]>(2*image.shape[0])):
        return('rectangular')
    image = cv2.resize(image, (224,224))
    if format == 'BGR':
        ## return BGR format array
        return image
    ## cv2.imdecode converted array into BGR format , convert it to RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # return the image
    return image

In [10]:
def photo(update, context):
    vgg_prediction = []
    inception_prediction = []
    xception_prediction = []
    user = update.message.from_user
    photo_file = update.message.photo[-1].get_file().file_path
    # photo_file.download('user_photo.jpg') 
    img = image_url_to_numpy_array_urllib(photo_file)
    if img == 'rectangular':
      message = "What a great pic, too bad that it is too rectangular for me to process, let's try squarer pic :)"
      context.bot.send_message(chat_id=update.effective_chat.id, text=message)
      return 0 
    img = img.reshape(1,224,224,3)
    vgg_model = load_model('vgg_model.h5')
    inception_model = load_model('inception_model.h5')
    xception_model = load_model('xceptio_model.h5')
    y_pred_vgg = vgg_model.predict(img)
    y_pred_inception = inception_model.predict(img)
    y_pred_xception = xception_model.predict(img)
    col_dict = {0: '#animal',
                1: '#human',
                2: '#mountain',
                3: '#nature',
                4: '#outdoors',
                5: '#person',
                6: '#plant',
                7: '#sea',
                8: '#tree',
                9: '#water'}
    
    for i in range(10):
        if y_pred_vgg[0][i] > 0.5:
            vgg_prediction.append(col_dict[i])
        if y_pred_inception[0][i] > 0:
            inception_prediction.append(col_dict[i])
        if y_pred_xception[0][i] > 0:
            xception_prediction.append(col_dict[i])
    
    if len(y_pred_vgg) > 0:
      vgg_message = 'VGG16: ' + ' '.join(vgg_prediction)
    else:
      vgg_message = "VGG16 can't recognize the pic, let's try one more..."

    if len(y_pred_inception) > 0:
      inception_message = 'INCEPTION: ' + ' '.join(inception_prediction)
    else:
      inception_message = "INCEPTION can't recognize the pic, let's try one more..."

    if len(y_pred_xception) > 0:
      xception_message = 'XCEPTION: ' + ' '.join(xception_prediction)
    else:
      xception_message = "XCEPTION can't recognize the pic, let's try one more..."

    message = vgg_message + '\n' + inception_message + '\n' + xception_message

    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text=message)

In [11]:
echo_handler = MessageHandler(Filters.photo & (~Filters.command), photo)
dispatcher.add_handler(echo_handler)

In [12]:
updater.start_polling()

In [13]:
# updater.stop()